In [35]:
# generate a RAG demo based on python llamaindex 
import os
from pathlib import Path

# LlamaIndex core
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

# initialize llm: support for ollma or openai compatible llm
from llama_index.llms.ollama import Ollama

# init data from document
documents_path = [r'C:\works\codes\GIT\github\model_validation\prd\AI_model_validation_report_GXS_Agent_Requirement.pdf']

In [36]:
# Configure backend; support 'deepseek' via OpenAI-compatible API, or 'openai', or local 'ollama' (default)
LLM_BACKEND = "ollama"

# default: ollama, with connectivity probe and fallback to deepseek/openai if provided
ollama_model = "deepseek-v3.1:671b-cloud"
# Support remote Ollama via env var OLLAMA_HOST (e.g., http://<ip>:11434)
ollama_host = "http://localhost:11434" #"http://10.80.17.97:11434"
#llm = Ollama(model=ollama_model, base_url=ollama_host)
llm = Ollama(model=ollama_model, request_timeout=120.0)
print(f"[LLM] Backend: ollama, \r\nmodel: {ollama_model}, \r\nbase_url: {ollama_host}")


from llama_index.core.prompts import PromptTemplate

topic = "who you are?"
prompt = f"Please write a response related to {topic}."
result= llm.stream(prompt)
print(result)


[LLM] Backend: ollama, 
model: deepseek-v3.1:671b-cloud, 
base_url: http://localhost:11434


AttributeError: 'str' object has no attribute 'kwargs'

In [ ]:
# Embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

def build_index(data_dir: Path):
    # Read docs from a folder
    docs = SimpleDirectoryReader(input_files=data_dir).load_data()

    # Configure embedding model (small, fast)
    #Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
    Settings.embed_model = HuggingFaceEmbedding(model_name=r"C:\hf_models\all-MiniLM-L6-v2")
    
    # Configure LLM
    Settings.llm = llm

    # Optional: capture debug traces
    Settings.callback_manager = CallbackManager([LlamaDebugHandler()])

    # Build a simple vector index
    index = VectorStoreIndex.from_documents(docs)
    return index

index = build_index(documents_path)

# query info
query_engine = index.as_query_engine()
response = query_engine.query("Some question about the data should go here")
print(response)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 0.225182 seconds
    |_CBEventType.EMBEDDING -> 0.016055 seconds
**********
**********
Trace: query
    |_CBEventType.QUERY -> 4.71844 seconds
      |_CBEventType.RETRIEVE -> 0.012029 seconds
        |_CBEventType.EMBEDDING -> 0.011031 seconds
      |_CBEventType.SYNTHESIZE -> 4.705411 seconds
        |_CBEventType.TEMPLATING -> 0.0 seconds
        |_CBEventType.LLM -> 4.615292 seconds
        |_CBEventType.EXCEPTION -> 0.0 seconds
**********


ValueError: "ChatResponse" object has no field "usage"

In [ ]:

def query_loop(index):
    print("RAG demo ready. Ask questions about your documents. Type 'exit' to quit.")
    query_engine = index.as_query_engine(similarity_top_k=4)
    while True:
        q = input("\nQuery> ").strip()
        if q.lower() in {"exit", "quit"}:
            break
        try:
            resp = query_engine.query(q)
            print("\nAnswer:\n", resp.response)
            # Optional: show sources
            print("\nSources:")
            for n in resp.source_nodes:
                src = n.metadata.get("file_name", n.metadata.get("filename", "unknown"))
                print(f"- {src} (score={n.score:.3f})")
        except Exception as e:
            print("Error:", e)


In [ ]:

def main():
    base = Path(__file__).parent
    data_dir = base / "data"
    data_dir.mkdir(exist_ok=True)

    # Seed a sample doc if none exists
    sample = data_dir / "sample.txt"
    if not sample.exists():
        sample.write_text(
            "LlamaIndex is a data framework for building LLM applications.\n"
            "It provides readers, indexes, query engines, and tools for Retrieval Augmented Generation (RAG).\n"
            "This is a simple demo document."
        )

    index = build_index(data_dir)
    query_loop(index)

if __name__ == "__main__":
    main()